In [1]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import trends_helpers
import os
import pycountry

In [2]:
os.getcwd()

"g:\\Mi unidad\\BSE - DSDM\\Master's project\\refugees\\notebooks\\trends"

In [3]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [4]:
currencies.head()

,Country Code,Country Name,WHO Region,World Bank Income Group (2018),Currency Name,Currency Code
0,AFG,Afghanistan,EMR,Low,Afghanistan Afghani,AFN
1,ALB,Albania,EUR,Upper‐middle,Albanian Lek,ALL
2,DZA,Algeria,AFR,Upper‐middle,Algerian Dinar,DZD
3,AND,Andorra,EUR,High,Euro,EUR
4,AGO,Angola,AFR,Lower‐middle,Angolan Kwanza,AON


In [5]:
currencies.info(
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 6 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Country Code                    194 non-null    object
 1   Country Name                    194 non-null    object
 2   WHO Region                      194 non-null    object
 3   World Bank Income Group (2018)  194 non-null    object
 4   Currency Name                   194 non-null    object
 5   Currency Code                   194 non-null    object
dtypes: object(6)
memory usage: 9.2+ KB


In [6]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

# First search as it comes, then we can translate for each origin lang

141

In [20]:
keywords_list = [
 'Kip',
 'Comoros Franc',
 'Dominican Peso',
 'Som',
 'Kuna',
 'U.A.Emirates Dirham']

In [8]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [21]:
from pytrends.request import TrendReq
import time


# Set up the Google Trends API object
# pytrends = TrendReq()
# I'll define the object within the loop

# Initialize an empty DataFrame to store the results
results = pd.DataFrame()


# Loop through regions and keywords
for keyword in keywords_list:
    for i, country in enumerate(countries):    
        # pytrends = TrendReq(timeout=(10,25), proxies=proxy_list, retries=2, backoff_factor=0.1,    )

        pytrends = TrendReq(tz=360, timeout=(10, 25), retries=2, backoff_factor=0.5, 
                            requests_args={'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299'}})
        
# The timeout parameter sets the maximum number of seconds to wait for a response from the API 
# before raising a timeout error.
#The backoff_factor parameter is used for exponential backoff when retrying requests. 
# It specifies the factor by which the waiting time between retries increases on each retry. 
# For example, if backoff_factor is set to 0.4, the waiting time between retries will increase by 40% on
# each retry.
# The retries parameter sets the maximum number of times to retry a request if it fails with a 
# transient error (such as a connection error or server error). If the maximum number of retries 
# is exceeded, an exception will be raised.   
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(4)
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])



Searching: Kip in AF
Searching: Kip in AL
Searching: Kip in DZ
Searching: Kip in AD
Searching: Kip in AO
Searching: Kip in AG
Searching: Kip in AR
Searching: Kip in AM
Searching: Kip in AU
Searching: Kip in AT
Searching: Kip in AZ
Searching: Kip in BS
Searching: Kip in BH
Searching: Kip in BD
Searching: Kip in BB
Searching: Kip in BY
Searching: Kip in BE
Searching: Kip in BZ
Searching: Kip in BJ
Searching: Kip in BT
Searching: Kip in BO
Searching: Kip in BA
Searching: Kip in BW
Searching: Kip in BR
Searching: Kip in BN
Searching: Kip in BG
Searching: Kip in BF
Searching: Kip in BI
Searching: Kip in CV
Searching: Kip in KH
Searching: Kip in CM
Searching: Kip in CA
Searching: Kip in CF
Searching: Kip in TD
Searching: Kip in CL
Searching: Kip in CN
Searching: Kip in CO
Searching: Kip in KM
Searching: Kip in CG
Searching: Kip in CK
Searching: Kip in CR
Searching: Kip in CI
Searching: Kip in HR
Searching: Kip in CU
Searching: Kip in CY
Searching: Kip in CZ
Searching: Kip in KP
Searching: Ki

In [22]:
results

,date,trends_index,keyword,region
0,2004-01-01,0,Kip,AF
1,2004-02-01,100,Kip,AF
2,2004-03-01,0,Kip,AF
3,2004-04-01,0,Kip,AF
4,2004-05-01,0,Kip,AF
...,...,...,...,...
228,2023-01-01,1,Kuna,ZW
229,2023-02-01,1,Kuna,ZW
230,2023-03-01,0,Kuna,ZW
231,2023-04-01,0,Kuna,ZW


In [23]:
results.to_csv("currency_partial results/currency39.csv")